# Update the image paths for the extracted features from Jan 29, 2018
March 27, 2018

Continue the work from `find_original_images.ipynb`. Use Keras to recompute the hashes of all my images, and see if I can find all of the original 1014 that I trained the model on.

In [1]:
import numpy as np
import pandas as pd
import imagehash
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#Solution to importing modules from a parallel directory from StackOverflow:
#https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from src.image_manager import ImageManager

## Create an ImageManager from my previously saved file database (saved as .csv's), then get a copy of its data frame
See `more_image_manager_and_scraper.ipynb` from Feb 5, 2018

In [28]:
base_directory = '../tree_photos/'
images_path = '../data/image_log_20180205.csv'
syncs_path = '../data/image_syncs_20180205.csv'

In [32]:
manager = ImageManager(base_directory, images_path, syncs_path)

In [33]:
image_df = manager.image_df.copy()
image_df.head()

,p_hash,filename,folder,time_added,time_verified
0,9bb759cb29b01a25,image_bigleaf_maple_57.png,acer_macrophyllum,2018-02-05 15:00:09.809644,2018-02-05 15:00:09.809644
1,99c20b3b74d53de0,image_bigleaf_maple_43.png,acer_macrophyllum,2018-02-05 15:00:09.819089,2018-02-05 15:00:09.819089
2,a1e1e0d4e646e765,image_big_leaf_maple_bark_48.png,acer_macrophyllum,2018-02-05 15:00:09.827989,2018-02-05 15:00:09.827989
3,abee94c19cb28b0e,image_acer_macrophylum_tree_49.png,acer_macrophyllum,2018-02-05 15:00:09.836027,2018-02-05 15:00:09.836027
4,d3c161631f2b4b4e,image_bigleaf_maple_94.png,acer_macrophyllum,2018-02-05 15:00:09.842604,2018-02-05 15:00:09.842604


In [34]:
manager.syncs_df

,time_started,time_completed,folders
0,2018-02-05 15:00:09.802240,2018-02-05 15:00:48.258705,"['acer_macrophyllum', 'betula_pendula', 'thuja..."
1,2018-02-05 21:54:57.298852,2018-02-05 21:56:21.355441,"['cedrus_libani', 'alnus_rubra', 'bad_images']"


## Open each image in `image_df` using Keras, compute its p_hash, and store the results in `image_df`

See `feature_extraction.ipynb` from Jan 29, 2018 to see how I originally computed the image hashes.

Method of applying a fumction to rows based on StackOverflow:
https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns

In [35]:
def p_hash_keras299(row):
    """Given a row of image_df, computes the p_hash of the image for that row
    after reading in the image using Keras with a target size of (299,299).
    """
    path = os.path.join(base_directory, row['folder'], row['filename'])
    img = image.load_img(path, target_size=(299,299))
    return imagehash.phash(img)

In [36]:
#Compute the keras299 p-hashes of all images in image_df
#Note that axis=1 means the function operates on rows rather than columns.
#From documentation of DataFrame.apply:
#"Objects passed to functions are Series objects having index either the
#DataFrame’s index (axis=0) or the columns (axis=1)."
keras299_hashes = image_df.apply(p_hash_keras299, axis = 1)
keras299_hashes.head()

0    9bb759cb29301aa5
1    99c20b3b74d53de0
2    a1e1e0d4e646e765
3    abee94c19cb28b0e
4    d3c161631f2b4b4e
dtype: object

In [37]:
image_df['p_hash_keras299'] = keras299_hashes
image_df.head()

,p_hash,filename,folder,time_added,time_verified,p_hash_keras299
0,9bb759cb29b01a25,image_bigleaf_maple_57.png,acer_macrophyllum,2018-02-05 15:00:09.809644,2018-02-05 15:00:09.809644,9bb759cb29301aa5
1,99c20b3b74d53de0,image_bigleaf_maple_43.png,acer_macrophyllum,2018-02-05 15:00:09.819089,2018-02-05 15:00:09.819089,99c20b3b74d53de0
2,a1e1e0d4e646e765,image_big_leaf_maple_bark_48.png,acer_macrophyllum,2018-02-05 15:00:09.827989,2018-02-05 15:00:09.827989,a1e1e0d4e646e765
3,abee94c19cb28b0e,image_acer_macrophylum_tree_49.png,acer_macrophyllum,2018-02-05 15:00:09.836027,2018-02-05 15:00:09.836027,abee94c19cb28b0e
4,d3c161631f2b4b4e,image_bigleaf_maple_94.png,acer_macrophyllum,2018-02-05 15:00:09.842604,2018-02-05 15:00:09.842604,d3c161631f2b4b4e


## Find images where the hashes don't match

It looks like there are 418 such images, or 7.26% of all 5760 images

In [38]:
image_df[image_df['p_hash'] != image_df['p_hash_keras299']].head()

,p_hash,filename,folder,time_added,time_verified,p_hash_keras299
0,9bb759cb29b01a25,image_bigleaf_maple_57.png,acer_macrophyllum,2018-02-05 15:00:09.809644,2018-02-05 15:00:09.809644,9bb759cb29301aa5
7,db7431cef4184de0,image_acer_macrophylum_tree_75.png,acer_macrophyllum,2018-02-05 15:00:09.866679,2018-02-05 15:00:09.866679,db7431cef6104de0
66,e2bc1e96d70632b1,image_acer_macrophylum_tree_62.png,acer_macrophyllum,2018-02-05 15:00:10.339740,2018-02-05 15:00:10.339740,e2bc1e86d70636b1
75,f69c29b94e4b9c24,image_bigleaf_maple_forest_33.png,acer_macrophyllum,2018-02-05 15:00:10.412334,2018-02-05 15:00:10.412334,f69c29b94e4f9424
78,b790d34b9e69c18a,image_acer_macrophylum_seeds_78.png,acer_macrophyllum,2018-02-05 15:00:10.433250,2018-02-05 15:00:10.433250,b790d34b9e69858a


In [39]:
image_df[image_df['p_hash'] != image_df['p_hash_keras299']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 5748
Data columns (total 6 columns):
p_hash             418 non-null object
filename           418 non-null object
folder             418 non-null object
time_added         418 non-null datetime64[ns]
time_verified      418 non-null datetime64[ns]
p_hash_keras299    418 non-null object
dtypes: datetime64[ns](2), object(4)
memory usage: 22.9+ KB


## Load the previously extracted features and convert the hash strings to imagehash objects
See `build_a_model.ipynb` from Jan 30, 2018

In [40]:
tree_features_df = pd.read_csv('../data/tree_images.csv', sep='|', index_col=0)
tree_features_df.head()

,p_hash,filename,species,tags,incv3_out_0,incv3_out_1,incv3_out_2,incv3_out_3,incv3_out_4,incv3_out_5,...,incv3_out_2038,incv3_out_2039,incv3_out_2040,incv3_out_2041,incv3_out_2042,incv3_out_2043,incv3_out_2044,incv3_out_2045,incv3_out_2046,incv3_out_2047
0,99a7465a1e99acb4,image_picea_sitchensis_27.png,picea_sitchensis,NaN,13.995308,1.316715,0.279829,67.839417,43.733177,38.206165,...,160.387985,1.176492,45.983730,10.465877,22.118896,3.226861,29.401524,0.486684,0.000000,15.634432
1,93d12f8e712ef068,image_picea_sitchensis_33.png,picea_sitchensis,NaN,7.449574,0.417233,0.447665,6.178374,2.092985,6.251201,...,2.649559,0.523397,0.392520,7.703988,13.842188,12.907540,1.112756,0.000000,5.436511,1.154409
2,cce3482eb991533b,image_picea_sitchensis_6.png,picea_sitchensis,NaN,0.708871,0.640444,1.305804,9.565918,15.157275,1.787687,...,15.114614,0.000000,0.066235,2.280773,8.082823,1.939496,0.041136,0.000000,0.000000,0.034437
3,c119991c4fcf1cda,image_picea_sitchensis_in_winter_12.png,picea_sitchensis,NaN,7.138457,1.225646,15.940742,42.644821,23.604031,13.818824,...,49.895081,0.000000,7.600242,3.260593,13.135204,12.249097,17.588070,0.000000,0.000000,0.000000
4,e3c3dab100fdc29c,image_sitka_spruce_59.png,picea_sitchensis,NaN,0.030419,2.616714,1.435360,52.145359,5.377345,5.631077,...,13.391465,21.970516,5.217728,8.282942,0.788009,0.591635,35.845409,0.000000,0.000000,2.780651


In [41]:
tree_features_df.loc[:,'p_hash'] = tree_features_df['p_hash'].apply(imagehash.hex_to_hash)
tree_features_df.loc[0,'p_hash']

array([[ True, False, False,  True,  True, False, False,  True],
       [ True, False,  True, False, False,  True,  True,  True],
       [False,  True, False, False, False,  True,  True, False],
       [False,  True, False,  True,  True, False,  True, False],
       [False, False, False,  True,  True,  True,  True, False],
       [ True, False, False,  True,  True, False, False,  True],
       [ True, False,  True, False,  True,  True, False, False],
       [ True, False,  True,  True, False,  True, False, False]], dtype=bool)

In [42]:
tree_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 0 to 1013
Columns: 2052 entries, p_hash to incv3_out_2047
dtypes: float64(2049), object(3)
memory usage: 15.9+ MB


## Find images in image_df whose keras299 hashes match those of my original 1014 images

In [43]:
image_df['p_hash_keras299'].isin(tree_features_df['p_hash']).describe()

count      5760
unique        2
top       False
freq       4745
Name: p_hash_keras299, dtype: object

In [44]:
#See how many images match
5760 - 4745

1015

### Great, now I have one extra instead of 64 missing! WTF?!
Perhaps there are two (or more...) images in image_df that have the same keras299 p_hash even though they have different PIL p_hashes?

Um, yes, it looks like there are 42 repeats:

In [45]:
image_df['p_hash_keras299'].describe()

count                 5760
unique                5718
top       97b2784de2cc6139
freq                     2
Name: p_hash_keras299, dtype: object

In [46]:
image_df[image_df['p_hash_keras299'] == imagehash.hex_to_hash('97b2784de2cc6139')]

,p_hash,filename,folder,time_added,time_verified,p_hash_keras299
2410,d3f79d0836e20dd0,image_cedrus_libani_49.png,cedrus_libani,2018-02-05 15:00:29.658666,2018-02-05 15:00:29.658666,97b2784de2cc6139
5063,97b2784de2cc6139,image_cedrus_libani_49.png,cedrus_libani,2018-02-05 21:29:27.829710,2018-02-05 21:55:26.950389,97b2784de2cc6139


### Ugh, why does the same file have two different p_hashes?!
Is there a bug in my ImageManager code?

Um, no, I think it was a bug in my ImageScraper code -- it was likely overwriting files it shouldn't have. See below.

In [26]:
image_df[image_df['p_hash_keras299'] == imagehash.hex_to_hash('d3f79d0836e20dd0')]
#So none of the keras299 hashes match the original p_hash of this image

,p_hash,filename,folder,time_added,time_verified,p_hash_keras299


In [24]:
image_df['filename'].describe()
#So it looks like there are 40 repeated filenames

count                           5760
unique                          5720
top       image_cedrus_libani_94.png
freq                               2
Name: filename, dtype: object

In [25]:
image_df[image_df['filename'] == 'image_cedrus_libani_94.png']

,p_hash,filename,folder,time_added,time_verified,p_hash_keras299
2330,e9e9c1178d1c965a,image_cedrus_libani_94.png,cedrus_libani,1517842828981828000,1517842828981828000,b293257039ce197e
4604,b293257039ce197e,image_cedrus_libani_94.png,cedrus_libani,1517866154904047000,1517867698086015000,b293257039ce197e


In [27]:
image_df[image_df['p_hash_keras299'] == imagehash.hex_to_hash('e9e9c1178d1c965a')]
#So none of the keras299 hashes match the original p_hash of this image

,p_hash,filename,folder,time_added,time_verified,p_hash_keras299


In [47]:
os.path.join('yak', '.txt')

'yak/.txt'

## Ok, here's what I think happened:

(written Fri May 11, 2018)

My ImageScraper code originally did not check whether a file existed before it wrote to disk. It worked like this: For every image it found during a web search, it checked whether the hash of that image matched a hash that was already in the dataframe. If so, that means (supposedly) that we already have the same image, so we don't need to save it. If not, that means (supposedly) that we found a new image, so we need to save it; thus, the image scraper would write a file named `image_{search_terms}_{i}.png`, where `i` is the number of images it has found so far. However, it did **not** check whether a file with this name already existed, which could easily happen if a previous search found a *different* image but used this same file name. Therefore, whenever this situation occurred, we would overwrite an old image with the new image, but we would still have the hash value for the old image stored in the dataframe. BAD!

Ok, so now, what would happen when we re-sync the (corrupted) dataframe to the new files? The dataframe, which has not been altered yet, is now "corrupted" in the sense that some of the primary keys (hash values) now map to the wrong images, because the correct images were overwritten. In other words, some of the image files listed in the dataframe will have different hash values than the dataframe says they have.

The first thing that happens in `.sync_images()` is that the dictionary gets synced to the current dataframe. That is, the existing dictionary is erased, and we create a new dictionary mapping the hash values in the current dataframe to the corresponding filename listed in the dataframe. So now, some of these hashes are mapping to an image file that does not actually have that hash. Consider a hypothetical scenario.

The dataframe originally stored image A with filename A and hash A. The image scraper overwrote filename A with image B. So now filename A stores image B, which has hash B, but the dataframe lists its hash as hash A. Now suppose we are syncing, and we come to filename A. The image manager computes the hash of filename A, which is hash B. There are two possibilities:

1. Hash B is not in the dataframe. In this case, the image manager adds a new row to the dataframe with hash B and filename A, and it adds the corresponding item to the dictionary. The dataframe now has two rows with filename A, the original one with hash A (which is now incorrect), and the new one with hash B (which is correct).

2. Hash B is already in the dataframe. How could this happen? Suppose that after the image scraper found the new image B, it found the same image B again later. Since we still wouldn't have had hash B stored in the dataframe, the scraper would have written image B again to a different file, say filename C. In this case, if filename C already existed, then it would be overwritten and we would get *yet another* incorrect filename/hash pair. If filename C did *not* already exist, then we would get a new file, filename C, with hash B.
 
On the other hand, suppose the image manager comes across a different image file, say filename D, which has hash A. Can this happen? This *shouldn't* have happened if the image scraper was checking whether the hash value existed in the dataframe before saving the image. But if I ran the code before I added that feature, I suppose it's possible. In that case, the image manager would store both filename A and filename D in the dictionary, and the order would depend on which one it encountered first.